In [8]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def normalize_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter')
        df = df.iloc[:,0].str.split(",",expand=True)
    else:
        df = pd.read_csv(folder+filename)


    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:] #row,col
        df_results = df_results.replace("",0)
        df_results = df_results.fillna(0)
        df_results = df_results.astype(int)

        df_results.columns = new_header
        df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        df.iloc[index+2:index+18,1:] = df_results_norm
        print("processed...",i)
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1])
    return(df)

def zscore_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        df = df.iloc[:,0].str.split(",",expand=True)
    else:
        df = pd.read_csv(folder+filename)
        
    df_initial = df
    df_results_index = df[df.iloc[:,0].str.contains('Results', na=False)].index
    normalized_dataframe = pd.DataFrame([])
    # print("DF",df, "INDEX",df_results_index[0])
    for i, index in enumerate(df_results_index):
        dateset_name = df.iloc[index,0]
        new_header = df.iloc[index+1,1:]
        df_results = df.iloc[index+2:index+18,1:] #row,col
        # print("i and index",i,index, dateset_name, df_results)
        df_results = df_results.replace("",0)
        df_results = df_results.fillna(0)
        df_results = df_results.astype(int)

        df_results.columns = new_header
        df_min = df_results.stack().min()
        df_max = df_results.stack().max()
        df_range = df_max-df_min
        df_results_norm = (df_results - df_min)/df_range
        
        df_mean = df_results_norm.stack().mean()
        df_std = df_results_norm.stack().std()
        df_results_zscore = (df_results_norm - df_mean)/df_std
        df.iloc[index+2:index+18,1:] = df_results_zscore
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_zscore."+filename.split(".")[-1])

    return(df)


repo = git.Repo('.', search_parent_directories=True)
folder = repo.working_tree_dir +"\\yeast-data\\"

plate_list = os.listdir(folder)
print(plate_list,folder)

for file in plate_list:
    if "plate-data" in file:
        normalize_plate384(folder,file)
        zscore_plate384(folder,file)

['archive', 'completed', 'plate-data-20230322_Surf1_plate1to5.csv', 'plate-data-20230322_Surf1_plate6to12.csv', 'plate-data-20230330_SURF1(batch 2).csv', 'plate-data-20230412_SURF1_Last16plates.csv', 'plate-data-ReFRAME PGAP3_Plate 28.csv'] C:\Users\Van\Desktop\vancanwin\yeast-data\
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230322_Surf1_plate1to5.csv
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230322_Surf1_plate1to5.csv
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230322_Surf1_plate6to12.csv
processed... 0
processed... 1
processed... 2
processed... 3
processed... 4
processed... 5
processed... 6
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230322_Surf1_plate6to12.csv
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230330_SURF1(batch 2).csv
processed... 0
processed... 1
processed... 2
processed... 3
processed... 4
processed... 5
processed... 6
processed... 7
processed... 8
processed... 9
processed... 10
processed... 11
processed... 12
pr

C:\Users\Van\AppData\Local\Temp\ipykernel_17956\2932497047.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter')
C:\Users\Van\AppData\Local\Temp\ipykernel_17956\2932497047.py:54: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
C:\Users\Van\AppData\Local\Temp\ipykernel_17956\2932497047.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid thi

C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230412_SURF1_Last16plates.csv
processed... 0
processed... 1
processed... 2
processed... 3
processed... 4
processed... 5
processed... 6
processed... 7
processed... 8
processed... 9
processed... 10
processed... 11
processed... 12
processed... 13
processed... 14
processed... 15
processed... 16
      1    2    3    4    5    6    7    8    9    10  ...   15   16   17  \
559  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
560  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
561  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
562  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
563  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
564  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
565  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
566  0.0  0.0  0.0  0.0  0.0  0.0  0.

C:\Users\Van\AppData\Local\Temp\ipykernel_17956\2932497047.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter')
C:\Users\Van\AppData\Local\Temp\ipykernel_17956\2932497047.py:54: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer